In [14]:
import os

In [15]:
%pwd

'e:\\MLOPS\\Data-Science-Project'

In [16]:
os.chdir("../")
%pwd

'e:\\MLOPS'

In [17]:
import pandas as pd

data=pd.read_csv("artifacts/data_ingestion/winequality-red.csv")
data.head()

FileNotFoundError: [Errno 2] No such file or directory: 'artifacts/data_ingestion/winequality-red.csv'

In [18]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1599 entries, 0 to 1598
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         1599 non-null   float64
 1   volatile acidity      1599 non-null   float64
 2   citric acid           1599 non-null   float64
 3   residual sugar        1599 non-null   float64
 4   chlorides             1599 non-null   float64
 5   free sulfur dioxide   1599 non-null   float64
 6   total sulfur dioxide  1599 non-null   float64
 7   density               1599 non-null   float64
 8   pH                    1599 non-null   float64
 9   sulphates             1599 non-null   float64
 10  alcohol               1599 non-null   float64
 11  quality               1599 non-null   int64  
dtypes: float64(11), int64(1)
memory usage: 150.0 KB


In [19]:
data.isnull().sum()

fixed acidity           0
volatile acidity        0
citric acid             0
residual sugar          0
chlorides               0
free sulfur dioxide     0
total sulfur dioxide    0
density                 0
pH                      0
sulphates               0
alcohol                 0
quality                 0
dtype: int64

In [20]:
data.shape

(1599, 12)

In [21]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataValidationConfig:
    root_dir:Path
    STATUS_FILE:str
    unzip_data_dir:Path
    all_schema:dict

In [22]:
from src.datascience.constants import *
from src.datascience.utils.common import read_yaml, create_directories

In [23]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_validation_config(self) -> DataValidationConfig:
        config = self.config.data_validation
        schema = self.schema.COLUMNS

        create_directories([config.root_dir])

        data_validation_config = DataValidationConfig(
            root_dir=config.root_dir,
            STATUS_FILE=config.STATUS_FILE,
            unzip_data_dir = config.unzip_data_dir,
            all_schema=schema,
        )

        return data_validation_config

In [24]:
import os
from src.datascience import logger

In [25]:
class DataValidation:
    def __init__(self, config: DataValidationConfig):
        self.config = config

    def validate_all_columns(self)-> bool:
        try:
            validation_status = None

            data = pd.read_csv(self.config.unzip_data_dir)
            all_cols = list(data.columns)

            all_schema = self.config.all_schema.keys()

            
            for col in all_cols:
                if col not in all_schema:
                    validation_status = False
                    with open(self.config.STATUS_FILE, 'w') as f:
                        f.write(f"Validation status: {validation_status}")
                else:
                    validation_status = True
                    with open(self.config.STATUS_FILE, 'w') as f:
                        f.write(f"Validation status: {validation_status}")

            return validation_status
        
        except Exception as e:
            raise e

    

In [28]:
import os
from pathlib import Path

# Check current directory
print(f"Current directory: {os.getcwd()}")

# Navigate to project root if needed
if Path.cwd().name == "research":
    os.chdir("../")
    print(f"Changed to: {os.getcwd()}")

# Verify config files exist
print(f"config.yaml exists: {Path('config/config.yaml').exists()}")

Current directory: e:\MLOPS
config.yaml exists: False


In [29]:
import os
from pathlib import Path

# Check where we are
print(f"Current directory: {os.getcwd()}")

# Look for the Data-Science-Project folder
project_path = Path("e:/MLOPS/Data-Science-Project")
print(f"Project exists: {project_path.exists()}")

# Change to project directory
if project_path.exists():
    os.chdir(project_path)
    print(f"Changed to: {os.getcwd()}")
    print(f"config.yaml exists: {Path('config/config.yaml').exists()}")
else:
    # Try to find it
    print("Searching for project...")
    for item in Path("e:/MLOPS").glob("*"):
        print(item)

Current directory: e:\MLOPS
Project exists: True
Changed to: e:\MLOPS\Data-Science-Project
config.yaml exists: True


In [30]:
try:
    config = ConfigurationManager()
    data_validation_config = config.get_data_validation_config()
    data_validation = DataValidation(config=data_validation_config)
    data_validation.validate_all_columns()
except Exception as e:
    raise e

[2026-02-02 05:21:00,078]: yaml file: config\config.yaml loaded successfully: common: yaml file: config\config.yaml loaded successfully]
[2026-02-02 05:21:00,082]: yaml file: config\params.yaml loaded successfully: common: yaml file: config\params.yaml loaded successfully]
[2026-02-02 05:21:00,104]: yaml file: config\schema.yaml loaded successfully: common: yaml file: config\schema.yaml loaded successfully]
[2026-02-02 05:21:00,106]: created directory at: artifacts: common: created directory at: artifacts]
[2026-02-02 05:21:00,108]: created directory at: artifacts/data_validation: common: created directory at: artifacts/data_validation]
